# Import libraries

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(15,10)})
import warnings
warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
!pip install -q datasist
from datasist.structdata import detect_outliers
from imblearn.over_sampling import RandomOverSampler
from category_encoders.binary import BinaryEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

- We have missing values

In [4]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


# Dealing with missing values

In [5]:
df.isna().sum() / len(df)*100

Date              0.000000
Location          0.000000
MinTemp           1.020899
MaxTemp           0.866905
Rainfall          2.241853
Evaporation      43.166506
Sunshine         48.009762
WindGustDir       7.098859
WindGustSpeed     7.055548
WindDir9am        7.263853
WindDir3pm        2.906641
WindSpeed9am      1.214767
WindSpeed3pm      2.105046
Humidity9am       1.824557
Humidity3pm       3.098446
Pressure9am      10.356799
Pressure3pm      10.331363
Cloud9am         38.421559
Cloud3pm         40.807095
Temp9am           1.214767
Temp3pm           2.481094
RainToday         2.241853
RainTomorrow      2.245978
dtype: float64

In [6]:
df.isna().sum() / len(df)*100

Date              0.000000
Location          0.000000
MinTemp           1.020899
MaxTemp           0.866905
Rainfall          2.241853
Evaporation      43.166506
Sunshine         48.009762
WindGustDir       7.098859
WindGustSpeed     7.055548
WindDir9am        7.263853
WindDir3pm        2.906641
WindSpeed9am      1.214767
WindSpeed3pm      2.105046
Humidity9am       1.824557
Humidity3pm       3.098446
Pressure9am      10.356799
Pressure3pm      10.331363
Cloud9am         38.421559
Cloud3pm         40.807095
Temp9am           1.214767
Temp3pm           2.481094
RainToday         2.241853
RainTomorrow      2.245978
dtype: float64

In [7]:
# let's drop na values in location
df.dropna(axis=0, subset=['Location'], inplace=True)

## Categories data fill null values

In [8]:
cat_imputer = SimpleImputer(strategy='most_frequent')
obj_cols = df.columns[df.dtypes=='object']
df[obj_cols] = cat_imputer.fit_transform(df[obj_cols])

## Numerical data fill null values

In [12]:
num_imputer = KNNImputer(n_neighbors=3)
num_cols = df.columns[df.dtypes!='object']
df[num_cols] = num_imputer.fit_transform(df[num_cols])

In [13]:
df.isna().sum() / len(df)*100

Date             0.0
Location         0.0
MinTemp          0.0
MaxTemp          0.0
Rainfall         0.0
Evaporation      0.0
Sunshine         0.0
WindGustDir      0.0
WindGustSpeed    0.0
WindDir9am       0.0
WindDir3pm       0.0
WindSpeed9am     0.0
WindSpeed3pm     0.0
Humidity9am      0.0
Humidity3pm      0.0
Pressure9am      0.0
Pressure3pm      0.0
Cloud9am         0.0
Cloud3pm         0.0
Temp9am          0.0
Temp3pm          0.0
RainToday        0.0
RainTomorrow     0.0
dtype: float64

no missing values

# Dealing with date

In [15]:
# from date convert to day, month
df['Date'] = pd.to_datetime(df['Date'])
df['day'] = df['Date'].dt.day
df['month'] = df['Date'].dt.month

In [16]:
# drop date
df.drop('Date', axis=1, inplace=True)

## label encoding

In [17]:
encoder = BinaryEncoder()
encode_cols = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
encoder_df = encoder.fit_transform(df[encode_cols])
df[encoder_df.columns] = encoder_df

In [18]:
df.drop(encode_cols, axis=1, inplace=True)

In [19]:
df['RainToday'] = df['RainToday'].apply(lambda s : 1 if s=='yes' or s=='Yes' else 0)

In [20]:
df['RainToday'].value_counts()

0    113580
1     31880
Name: RainToday, dtype: int64

In [21]:
df['RainTomorrow'] = df['RainTomorrow'].apply(lambda s : 1 if s=='Yes' or s=='Y' else 0)

In [22]:
df['RainTomorrow'].value_counts()

0    113583
1     31877
Name: RainTomorrow, dtype: int64

In [23]:
df.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow', 'day', 'month',
       'Location_0', 'Location_1', 'Location_2', 'Location_3', 'Location_4',
       'Location_5', 'WindGustDir_0', 'WindGustDir_1', 'WindGustDir_2',
       'WindGustDir_3', 'WindGustDir_4', 'WindDir9am_0', 'WindDir9am_1',
       'WindDir9am_2', 'WindDir9am_3', 'WindDir9am_4', 'WindDir3pm_0',
       'WindDir3pm_1', 'WindDir3pm_2', 'WindDir3pm_3', 'WindDir3pm_4'],
      dtype='object')

# Detect outliers

In [24]:
for i in num_cols:
    Q1 = df[i].quantile(0.25)
    Q3 = df[i].quantile(0.75)
    IQR = Q3 - Q1

    upper = Q3 + IQR * 1.5
    down = Q1 - IQR * 1.5

    values = df[i].apply(lambda val:val if val < upper and val > down else 0)
    
    indecs = detect_outliers(df, n=0, features=[i])
    df.loc[indecs, i] = values.mean()

In [25]:
x = df.drop('RainTomorrow', axis=1)
y = df['RainTomorrow']

In [26]:
y.value_counts()

0    113583
1     31877
Name: RainTomorrow, dtype: int64

# Split data

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

# Over sample the data

In [30]:
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(x_train, y_train)

# Scale the data

In [31]:
scaler = RobustScaler()
X_resampled = scaler.fit_transform(X_resampled)
x_test = scaler.transform(x_test)

# Apply model
### from analysis the data we notice 
- not all features linearly correlated so, linear models
- there is some Overlapping in data so, exclude K neighbor classifier
- algo with tree based is  a good choice for this data

### Let's check out tree based models

In [32]:
models = {
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(),
    "XGB": XGBClassifier()
}

In [33]:
for name, model in models.items():
    print(f'Training Model {name} \n--------------')
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(x_test)
    print(f'Training Accuracy: {model.score(X_resampled, y_resampled)}')
    print(f'Testing Accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'Testing Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
    print('-'*30)

Training Model DT 
--------------
Training Accuracy: 0.9999889947835274
Testing Accuracy: 0.7776708373435997
Testing Confusion Matrix: 
[[19456  3261]
 [ 3207  3168]]
------------------------------
Training Model RF 
--------------
Training Accuracy: 0.9999889947835274
Testing Accuracy: 0.8538429808882166
Testing Confusion Matrix: 
[[21274  1443]
 [ 2809  3566]]
------------------------------
Training Model XGB 
--------------
[06:19:25] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training Accuracy: 0.8540103008826183
Testing Accuracy: 0.8114945689536642
Testing Confusion Matrix: 
[[18691  4026]
 [ 1458  4917]]
------------------------------
